In [1]:
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, PointStruct

import pandas as pd

In [2]:
qdrant_client = QdrantClient(
    url="http://localhost:6333"
)

qdrant_client.create_collection(
    collection_name="Amazon-items-collection-00",
    vectors_config=VectorParams(size=1536, distance=Distance.COSINE))

True

In [3]:
df_items = pd.read_json('../data/meta_Electronics_2022_2023_with_category_ratings_100_sample_1000.jsonl', lines=True)

### Concatenate title and featues

In [4]:
def preprocess_data(row):
    return f"{row['title']} {''.join(row['features'])}"

In [5]:
df_items["preprocessed_data"] = df_items.apply(preprocess_data, axis=1)

In [6]:
# df_items.head(2)

### Sample 50 items 

In [7]:
df_sample = df_items.sample(50, random_state=42)

### Embedding function

In [8]:
import openai
from dotenv import load_dotenv
import os

load_dotenv()
openai.api_key = os.getenv("OPEN_AI_KEY")

In [9]:
def get_embedding(text, model="text-embedding-3-small"):
    response = openai.embeddings.create(
        input=[text],
        model=model
    )
    return response.data[0].embedding

### Embed data

In [10]:
data_to_emded = df_sample["preprocessed_data"].tolist()
pointstructs = []
for i, data in enumerate(data_to_emded):
    embedding = get_embedding(data)
    pointstructs.append(
        PointStruct(
        id=i,
        vector=embedding,
        payload={"text": data}
    )
)

In [11]:
# pointstructs

### Write embedded data to Qdrant

In [12]:
qdrant_client.upsert(
    collection_name="Amazon-items-collection-00",
    wait=True,
    points=pointstructs
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

### Function for data retrieval

In [13]:
def retrieve_data(query):
    query_embedding = get_embedding(query)
    results = qdrant_client.query_points(
        collection_name="Amazon-items-collection-00",
        query=query_embedding,
        limit=10
    )
    return results

In [14]:
try:
    qdrant_client.get_collections()
    print("Qdrant is up!")
except Exception as e:
    print(f"Connection failed: {e}")

Qdrant is up!


In [15]:
retrieve_data("What earphones can I get?").points

[ScoredPoint(id=11, version=0, score=0.5329136, payload={'text': 'Wireless Earbuds Bluetooth 5.3 Headphones with 4-Mics Clear Call and ENC Noise Cancelling, Bluetooth Earbuds Touch Control Stereo Sound with LED Display, Waterproof Running Headphones for Workout Enjoy Bluetooth 5.3 and Fast Auto Pairing: Latest Bluetooth 5.3 technology achieves lower latency and stronger anti-interference. MD058A true bluetooth headphone offer audio delays much lower than 65ms. Feature reduces the number of disconnections of the ear buds in complex environments(places with many people, such as subways and buses). Adopting hall switch, after the first connection, it will connect with your device as soon as the charging case opens.4-Mic and ENC Call Noise Reduction: Each headset has 2 built-in microphones, equipped with Environmental Noise Reduction (ENC) technology that can suppress 80% of the interfering background noise in the calling environment, intelligently enhances human voicethrough the four micr